In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

# import filtered data

In [2]:
data = pd.read_csv('./ny_filtered.csv',index_col=0)
data.head()

C:\Users\User\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,year,month,day,weekday,hour,x1,y1,x2,y2
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,0.009436,2009,6,15,1,17,1331,851,1338,816
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,0.079696,2010,1,5,2,16,829,812,937,1084
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,0.013674,2011,8,18,4,0,926,1005,902,963
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,0.025340,2012,4,21,6,4,914,896,901,992
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,0.019470,2010,3,9,2,7,969,1030,1003,1091


# preprocessing(2)

In [3]:
data.drop(['x1','y1','x2','y2'],axis=1,inplace=True)

In [4]:
datax = data.drop(['fare_amount'],axis=1)
datay = data['fare_amount']

In [5]:
datax.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,year,month,day,weekday,hour
0,-73.844311,40.721319,-73.841610,40.712278,1,0.009436,2009,6,15,1,17
1,-74.016048,40.711303,-73.979268,40.782004,1,0.079696,2010,1,5,2,16
2,-73.982738,40.761270,-73.991242,40.750562,2,0.013674,2011,8,18,4,0
3,-73.987130,40.733143,-73.991567,40.758092,1,0.025340,2012,4,21,6,4
4,-73.968095,40.768008,-73.956655,40.783762,1,0.019470,2010,3,9,2,7


In [126]:
x_train,x_test,y_train,y_test = train_test_split(datax,datay,test_size=0.1,random_state=0)

# model training(XGBoost)

In [15]:
import xgboost as xgb

In [128]:
xgb_train=xgb.DMatrix(x_train,y_train)
xgb_test =xgb.DMatrix(x_test,y_test)

C:\Users\User\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [135]:
fit_para ={
'num_boost_round':1200,
'verbose_eval':100,
}
para = {
'learning_rate': 0.05,
'max_depth': 7,
'min_child_weight': 1,
'subsample': 0.8,
'colsample_bytree':0.8,
'booster':'gbtree',
'objective':'reg:linear',
'nthread':-1
}

In [136]:
model_jmda = xgb.train(dtrain=xgb_train,params=para,**fit_para,evals=[(xgb_train,'train'),(xgb_test,'test')])

[17:41:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:41:07] WARNING: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	train-rmse:11.9484	test-rmse:11.9289
[100]	train-rmse:2.81031	test-rmse:2.81854
[200]	train-rmse:2.71533	test-rmse:2.72954
[300]	train-rmse:2.66319	test-rmse:2.6826
[400]	train-rmse:2.6263	test-rmse:2.65036
[500]	train-rmse:2.60184	test-rmse:2.63045
[600]	train-rmse:2.58417	test-rmse:2.61676
[700]	train-rmse:2.56727	test-rmse:2.60479
[800]	train-rmse:2.5527	test-rmse:2.59457
[900]	train-rmse:2.54156	test-rmse:2.5882
[1000]	train-rmse:2.53065	test-rmse:2.58241
[1100]	train-rmse:2.52054	test-rmse:2.57752
[1199]	train-rmse:2.5118	test-rmse:2.57332


# model training(RF)

In [41]:
model_jmda = RandomForestRegressor(n_estimators=43,max_depth=13)
model_jmda.fit(x_train,y_train)
model_jmda.score(x_test,y_test)

0.863175862944648

In [46]:
for i in zip(model_jmda.feature_importances_,datax.columns):
    print(i)

(0.006449553401248837, 'pickup_longitude')
(0.004346030048779438, 'pickup_latitude')
(0.008176893360960034, 'dropoff_longitude')
(0.012321519787107132, 'dropoff_latitude')
(0.00018786464945402578, 'passenger_count')
(0.9266970842911727, 'distance')
(0.027807528876261468, 'year')
(0.0034891107948415738, 'month')
(0.0009290155895758395, 'day')
(0.001836373304837192, 'weekday')
(0.007759025895761809, 'hour')


# model training(DL)

In [50]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.preprocessing import MinMaxScaler

In [65]:
datax_nom = MinMaxScaler().fit_transform(datax)
x_train_1,x_test_1,y_train_1,y_test_1 = train_test_split(datax_nom,datay.values,test_size=0.05,random_state=0)

In [67]:
model_jmda_dl = Sequential()
model_jmda_dl.add(Dense(1024,activation='relu'))
model_jmda_dl.add(Dropout(0.2))
model_jmda_dl.add(Dense(256,activation='relu'))
model_jmda_dl.add(Dropout(0.2))
model_jmda_dl.add(Dense(64,activation='relu'))
model_jmda_dl.add(Dropout(0.2))
model_jmda_dl.add(Dense(16,activation='relu'))
model_jmda_dl.add(Dropout(0.2))
model_jmda_dl.add(Dense(1))
model_jmda_dl.compile(optimizer='adam',loss='mse')

In [68]:
model_jmda_dl.fit(x_train_1,y_train_1,batch_size=1000,epochs=30,validation_split=0.1)

Train on 4045523 samples, validate on 449503 samples
Epoch 1/30
4045523/4045523 [==============================] - 121s 30us/sample - loss: 16.7248 - val_loss: 9.6816
Epoch 2/30
4045523/4045523 [==============================] - 122s 30us/sample - loss: 13.0539 - val_loss: 8.7471
Epoch 3/30
4045523/4045523 [==============================] - 122s 30us/sample - loss: 11.6888 - val_loss: 8.5793
Epoch 4/30
4045523/4045523 [==============================] - 125s 31us/sample - loss: 10.9721 - val_loss: 8.5520
Epoch 5/30
4045523/4045523 [==============================] - 127s 31us/sample - loss: 10.5415 - val_loss: 8.2719
Epoch 6/30
4045523/4045523 [==============================] - 124s 31us/sample - loss: 10.3156 - val_loss: 7.9709
Epoch 7/30
4045523/4045523 [==============================] - 127s 31us/sample - loss: 10.1719 - val_loss: 8.2371
Epoch 8/30
4045523/4045523 [==============================] - 137s 34us/sample - loss: 10.0893 - val_loss: 8.0028
Epoch 9/30
4045523/4045523 [=======

# metrics

In [108]:
def Regression_evaluator(y_test, y_pred):
    # 平均絶対誤差 (MAE: Mean Absolute Error)
    from sklearn.metrics import mean_absolute_error
    print("平均絶対誤差：", mean_absolute_error(y_test, y_pred))
    # 平均二乗誤差 (MSE: Mean Squared Error)
    from sklearn.metrics import mean_squared_error
    print("平均絶対誤差：", mean_squared_error(y_test, y_pred))
    # 二乗平均平方根誤差 (RMSE: Root Mean Squared Error) -- sklearn には実装されていないのでnumpyを利用
    import numpy as np
    print("二乗平均平方根誤差：", np.sqrt(mean_squared_error(y_test, y_pred)))
    # 相関係数（Correlation)
    from sklearn.metrics import r2_score
    print("相関係数 (R)：", np.sqrt(r2_score(y_test, y_pred)))
    # 決定係数 (R2, R-squared, coefficient of determination)) 
    from sklearn.metrics import r2_score
    print("決定係数 (R2)：", r2_score(y_test, y_pred))

(1)Random forest

In [54]:
y_pred_rf = model_jmda.predict(x_test)
Regression_evaluator(y_test,y_pred_rf)

平均絶対誤差： 1.600803979776155
平均絶対誤差： 7.519675023805072
二乗平均平方根誤差： 2.7422025862078594
相関係数 (R)： 0.9290725821724846
決定係数 (R2)： 0.863175862944648


(2)deep learning

In [69]:
y_pred_dl = np.concatenate(model_jmda_dl.predict(x_test_1))
Regression_evaluator(y_test_1,y_pred_dl)

平均絶対誤差： 1.4903063957855491
平均絶対誤差： 7.344570505603727
二乗平均平方根誤差： 2.7100868077616496
相関係数 (R)： 0.9307856760005601
決定係数 (R2)： 0.8663619746478195


(3)XGBoost

In [137]:
xgb_test_x = xgb.DMatrix(x_test)
y_pred_xgb = model_jmda.predict(xgb_test_x)
Regression_evaluator(y_test,y_pred_xgb)

平均絶対誤差： 1.3855217148699803
平均絶対誤差： 6.622279446261966
二乗平均平方根誤差： 2.5733789939031455
相関係数 (R)： 0.9380748154277693
決定係数 (R2)： 0.8799843593398433
